# Titanic - Machine Learning from Disaster
Source:  
https://www.kaggle.com/competitions/titanic

### The Challenge
The sinking of the Titanic is one of the most infamous shipwrecks in history.

On April 15, 1912, during her maiden voyage, the widely considered “unsinkable” RMS Titanic sank after colliding with an iceberg. Unfortunately, there weren’t enough lifeboats for everyone onboard, resulting in the death of 1502 out of 2224 passengers and crew.

While there was some element of luck involved in surviving, it seems some groups of people were more likely to survive than others.

In this challenge, we ask you to build a predictive model that answers the question: “what sorts of people were more likely to survive?” using passenger data (i.e. name, age, gender, socio-economic class, etc).

### What Data Will I Use in This Competition?
In this competition, you have access to a Snowflake table called **PASSENGERS**.  
This table contains the **PASSENGER_ID** and a label-column **SURVIVED** which indicates whether a passenger sruvived or not.  
We know the survival status for 891 passengers but the status for the remaining 418 passengers is unknown and therefore missing in that table.

In addition, your team of smart datascientists already registered a couple of features in your **Snowflake Feature Store**.  
This feature store includes information about the name, age, gender, socio-economic class, etc. 

Use your datascience expertise to:
* Explore the existing Data
* Develop and Register new Features in the **Feature Store**
* Train and Register a Machine Learning Model in the **Model Registry**
* Create Scores for passengers with unknown survival status

### Evaluation
#### Goal
It is your job to predict if a passenger survived the sinking of the Titanic or not.  
For each passenger where the survival status is unknown, you must predict a 0 (died) or 1 (survived).

#### Metric
Your score is the percentage of passengers you correctly predict. This is known as accuracy.

# Imports

In [ ]:
# Snowpark Imports
from snowflake.snowpark.context import get_active_session
import snowflake.snowpark.functions as F
from snowflake.snowpark.functions import col, lit, when

# Snowpark ML
from snowflake.ml.modeling.impute import SimpleImputer
from snowflake.ml.modeling.preprocessing import OrdinalEncoder, OneHotEncoder, Normalizer
from snowflake.ml.modeling.pipeline import Pipeline
from snowflake.ml.modeling.xgboost import XGBClassifier
from snowflake.ml.modeling.model_selection import GridSearchCV
from snowflake.ml.registry import Registry
from snowflake.ml.feature_store import FeatureStore, FeatureView, CreationMode
from snowflake.cortex import Complete

# Other Imports
import matplotlib.pyplot as plt
import json
import streamlit as st
import plotly.express as px
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

# 2 - Set Up Environment

In [ ]:
# Retrieve the Session
session = get_active_session()

# Set context
session.use_schema('KAGGLE_TITANIC_CHALLENGE.DEVELOPMENT')

# Create reference to Feature Store
fs = FeatureStore(
    session=session, 
    database="KAGGLE_TITANIC_CHALLENGE", 
    name="DEVELOPMENT", 
    default_warehouse="COMPUTE_WH",
    creation_mode=CreationMode.CREATE_IF_NOT_EXIST
)

# Create reference to Model Registry
model_registry = Registry(
    session=session, 
    database_name=session.get_current_database(), 
    schema_name=session.get_current_schema()
)

# 3 - Data Exploration

In [ ]:
passengers = session.table('PASSENGER')
passengers.show()

In [ ]:
# Discover available entites, feature views and features
print('Entities:')
fs.list_entities().show()
entity = fs.get_entity(name="PASSENGER")

print('Feature Views:')
fs.list_feature_views().show()
kaggle_fv = fs.get_feature_view('PASSENGER_KAGGLE_FEATURES','V1')

print('Features in PASSENGER_KAGGLE_FEATURES:')
pd.DataFrame(list(kaggle_fv.feature_descs.items()), columns=['FEATURE_NAME', 'DESCRIPTION'])

In [ ]:
# Retrieve existing features for your data
titanic_df = fs.retrieve_feature_values(passengers, [kaggle_fv])
titanic_df.show()

In [ ]:
# Statistics for all columns in dataset (where SURVIVED is not missing)
train_df_summary = titanic_df.describe().to_pandas()
train_df_summary

That's a ton of information! What do these statistics mean? What should I do?  
Let's ask an LLM!

In [ ]:
#llm = 'mixtral-8x7b'
llm = 'llama3-70b'

prompt = f"""
I used Snowparks describe function to calculate count, mean, stddev, min and max per column.
PASSENGER_ID is unique and identifies each row. I want to predict the variable SURVIVED.
What feature engineering steps should I perform before building a machine learning model?
Make sure to explain your recommendations based on the data.
{train_df_summary.to_markdown()}
"""

response = Complete(llm, prompt)
st.markdown(response)

In [ ]:
# Create a figure and a set of subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(6, 3))

# Create a boxplot in the first subplot
ax1.boxplot(titanic_df[['AGE']].dropna().to_pandas())
ax1.set_title('Boxplot of AGE')
ax1.set_ylabel('Values')

# Create a boxplot in the second subplot
ax2.boxplot(titanic_df[['FARE']].dropna().to_pandas())
ax2.set_title('Boxplot of FARE')
ax2.set_ylabel('Values')

# Adjust layout to prevent overlap
plt.tight_layout()

In [ ]:
# Visualize variables in relation to survival probability
col1, col2 = st.columns(2)
col1.bar_chart(titanic_df.group_by('SEX').agg(F.avg('SURVIVED').as_('SURVIVAL_PROB')).to_pandas(), x='SEX', y='SURVIVAL_PROB')
col2.bar_chart(titanic_df.group_by('EMBARKED').agg(F.avg('SURVIVED').as_('SURVIVAL_PROB')).to_pandas(), x='EMBARKED', y='SURVIVAL_PROB')
col1.bar_chart(titanic_df.group_by('PCLASS').agg(F.avg('SURVIVED').as_('SURVIVAL_PROB')).to_pandas(), x='PCLASS', y='SURVIVAL_PROB')
col2.bar_chart(titanic_df.group_by('SIB_SP').agg(F.avg('SURVIVED').as_('SURVIVAL_PROB')).to_pandas(), x='SIB_SP', y='SURVIVAL_PROB')

# 4 - Feature Engineering

In [ ]:
# Features: Family Size
# We can imagine that large families will have more difficulties to evacuate, looking for theirs sisters/brothers/parents during the evacuation. 
# So, we to create a "FAMILY_SIZE" feature which is the sum of SIBSP , PARCH and 1 (including the passenger).
titanic_df = titanic_df.with_column('FAM_SIZE', col('SIB_SP') + col('PARCH') + 1)

# We further can create groups based on the FAMILY_SIZE
titanic_df = titanic_df.with_column('FAM_SIZE_CATEGORY', 
                                when(col('FAM_SIZE') == 1, 'SINGLE')
                                .when(col('FAM_SIZE') == 2, 'COUPLE')
                                .when(col('FAM_SIZE') >= 5, 'LARGE_FAMILY')
                                .otherwise('NORMAL_FAMILY'))

titanic_df[['FAM_SIZE','FAM_SIZE_CATEGORY','SURVIVED']].show(3)

# Analyze family sizes for training data
analysis_df = titanic_df.filter(col('SURVIVED').is_not_null()).group_by('FAM_SIZE_CATEGORY')
analysis_df = analysis_df.agg(F.count('FAM_SIZE_CATEGORY').as_('COUNT'), F.avg('SURVIVED').as_('SURVIVAL_PROB')).order_by('SURVIVAL_PROB')
analysis_df = analysis_df.to_pandas()

fig = px.scatter(analysis_df, x="COUNT", y="SURVIVAL_PROB", size="COUNT", color="FAM_SIZE_CATEGORY",
                 hover_name="FAM_SIZE_CATEGORY", title="Frequency of Family Sizes and Their Relation to Survival Probability",
                 labels={"COUNT": "Count of Titles", "SURVIVAL_PROB": "Survival Probability"},
                 size_max=60)

st.plotly_chart(fig, use_container_width=True)

In [ ]:
# Retrieve title from name
titanic_df = titanic_df.with_column('TITLE',F.trim(F.split(F.split(col('NAME'), F.lit(','))[1],F.lit('.'))[0]))
# Uppercase the title
titanic_df = titanic_df.with_column('TITLE',F.upper(col('TITLE')))
titanic_df[['NAME','TITLE','SURVIVED']].show(3)

# Combine rare titles
rare_titles = ['Lady', 'the Countess','Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona','Ms','Mme','Mlle']
rare_titles = [title.upper() for title in rare_titles]
titanic_df = titanic_df.with_column('TITLE', when(col('TITLE').isin(rare_titles), 'Rare').otherwise(col('TITLE')))

# Analyze titles
analysis_df = titanic_df.filter(col('SURVIVED').is_not_null()).group_by('TITLE')
analysis_df = analysis_df.agg(F.count('TITLE').as_('COUNT'), F.avg('SURVIVED').as_('SURVIVAL_PROB')).order_by('SURVIVAL_PROB')
analysis_df = analysis_df.to_pandas()

fig = px.scatter(analysis_df, x="COUNT", y="SURVIVAL_PROB", size="COUNT", color="TITLE",
                 hover_name="TITLE", title="Frequency of Titles and Their Relation to Survival Probability",
                 labels={"COUNT": "Count of Titles", "SURVIVAL_PROB": "Survival Probability"},
                 size_max=60)

st.plotly_chart(fig, use_container_width=True)


We are lazy so let's use an LLM to generate the feature descriptions based on the underlying SQL transformations of our Snowpark DataFrame.

In [ ]:
llm = 'llama3-70b'

prompt = f"""
You are provided with a SQL Query that derives features from existing columns.
Describe the features FAM_SIZE, FAM_SIZE_CATEGORY and TITLE.
The descriptions will be stored in a feature store, so make sure to return a JSON where the feature name is the key and the description is the value.
{titanic_df.queries['queries'][0]}
"""

llm_response = Complete(llm, prompt)
feature_descriptions = json.loads(llm_response.split('```')[1])
for key in feature_descriptions:
    feature_descriptions[key] = feature_descriptions[key].replace("'", '')
feature_descriptions

In [ ]:
# Create Feature View with Custom Features
custom_fv = FeatureView(
    name="PASSENGER_CUSTOM_FEATURES", 
    entities=[entity],
    feature_df=titanic_df[['PASSENGER_ID','TITLE','FAM_SIZE','FAM_SIZE_CATEGORY']],
    refresh_freq="1 minute",
    desc="Custom Passenger Features")

# Add descriptions for some features
custom_fv = custom_fv.attach_feature_desc(feature_descriptions)

custom_fv = fs.register_feature_view(
    feature_view=custom_fv, 
    version="V1", 
    block=True)

In [ ]:
spine_df = session.table('PASSENGER')

spine_df_train = spine_df.filter(col('SURVIVED').is_not_null())
print(f'Train dataset has {spine_df_train.count()} passengers.')
spine_df_train.show(3)

spine_df_test = spine_df.filter(col('SURVIVED').is_null())
print(f'Test dataset has {spine_df_test.count()} passengers.')
spine_df_test.show(3)

In [ ]:
# Generate the training dataset by retrieving the features
training_dataset = fs.generate_dataset(
    name="TITANIC_TRAINING_DATASET",
    spine_df=spine_df_train,
    features=[kaggle_fv,custom_fv],
    spine_label_cols=["SURVIVED"],
    desc="Training Data to train model to predict whether a passenger survived."
)

# Retrieve a Snowpark DataFrame from the registered Dataset
training_dataset_df = training_dataset.read.to_snowpark_dataframe().cache_result()
training_dataset_df.show(3)

# 5 - Modelling

In [ ]:
# Switch to a larger warehouse to speed up training
session.use_warehouse('TRAIN_WH')

In [ ]:
# DROP unused variables
# PASSENGER_ID -> just an artificial ID with no predictive value
# NAME -> people won't survive just because of their name and we extracted the title already
# CABIN -> too many missing values
# TICKET -> doesn't contain valuable information in its current form since it's a unique value per customer
training_dataset_df = training_dataset_df.drop(['PASSENGER_ID','NAME','CABIN','TICKET'])

# Impute Age by mean
si_age =  SimpleImputer(
    input_cols=['AGE','FARE'], 
    output_cols=['AGE_IMP','FARE_IMP'],
    strategy='mean',
    drop_input_cols=True
)

# Normalize Fare and Age
norm = Normalizer(
    input_cols=['AGE_IMP','FARE_IMP'],
    output_cols=['AGE_IMP_NORM','FARE_IMP_NORM'],
    drop_input_cols=True
)

# One-Hot-Encoding of categorical features
ohe = OneHotEncoder(
    input_cols=['SEX','EMBARKED','TITLE','FAM_SIZE_CATEGORY'], 
    output_cols=['SEX','EMBARKED','TITLE','FAM_SIZE_CATEGORY'],
    drop_input_cols=True
)

# Define the XGBoost model (incl. Hyperparameter Tuning)
label_cols = ['SURVIVED']
output_cols = ['SURVIVED_PREDICTION']

grid_search = GridSearchCV(
    estimator=XGBClassifier(random_state=42),
    param_grid={
        'n_estimators':[10, 50, 100],
        'max_depth': [2,4,8],
        'learning_rate':[.01, .03, .1],
    },
    n_jobs = -1,
    scoring="accuracy",
    label_cols=label_cols,
    output_cols=output_cols
)

# Build the pipeline
model_pipeline = Pipeline(
    steps=[
        ("IMPUTE",si_age),
        ("NORMALIZE",norm),
        ("ONE_HOT_ENCODE",ohe),
        ("GRIDSEARCH_XGBOOST",grid_search)
    ]
)

# Fit the pipeline to the training data
fitted_pipeline = model_pipeline.fit(training_dataset_df)

In [ ]:
# Switch back to smaller warehouse to save ressources
session.use_warehouse('COMPUTE_WH')

# 6 - Model Evaluation

In [ ]:
model_object = fitted_pipeline.to_sklearn().named_steps['GRIDSEARCH_XGBOOST']

# Get the Feature Importance
feature_importance = pd.DataFrame(
    zip(model_object.best_estimator_.feature_names_in_, model_object.best_estimator_.feature_importances_),
    columns=['FEATURE_NAME','IMPORTANCE']
).sort_values('IMPORTANCE', ascending=False)

fig = px.bar(feature_importance, x='FEATURE_NAME', y='IMPORTANCE')
st.plotly_chart(fig)

In [ ]:
# Build a dataframe for the gridsearch results
gs_results = model_object.cv_results_
n_estimators_val = []
learning_rate_val = []
max_depth_val = []
for param_dict in gs_results["params"]:
    n_estimators_val.append(param_dict["n_estimators"])
    learning_rate_val.append(param_dict["learning_rate"])
    max_depth_val.append(param_dict["max_depth"])
accuracy_val = gs_results["mean_test_score"]

gs_results_df = pd.DataFrame(data={
    "n_estimators":n_estimators_val,
    "learning_rate":learning_rate_val,
    "max_depth":max_depth_val,
    "accuracy":accuracy_val})

print(f'Number of Models: {len(gs_results_df)}')
print('Best Parameter Configuration:')
model_object.best_params_
print(f'Accuracy of best Model: {model_object.best_score_}')

# Create a 3D scatter plot to visualize impact of parameters on accuracy
fig = px.scatter_3d(gs_results_df, x='learning_rate', y='n_estimators', z='max_depth', color='accuracy',
                    labels={'accuracy': 'accuracy'})

# Update the layout to increase the size of the chart
fig.update_layout(
    width=1000,  # Set the desired width
    height=800   # Set the desired height
)

# Display the chart
st.plotly_chart(fig)

# 7 - Register Model

In [ ]:
# Register new model version
registered_model = model_registry.log_model(
    fitted_pipeline,
    model_name="TITANIC_SURVIVAL_MODEL",
    comment="Model trained using GridsearchCV in Snowpark to predict survival of Titanic passengers.",
    metrics={"accuracy": model_object.best_score_},
    conda_dependencies=['xgboost'],
    version_name='DEMO_VERSION'
)

# View available models
model_registry.show_models()

# View available versions
model_registry.get_model('TITANIC_SURVIVAL_MODEL').show_versions()

# 8 - Scoring Passengers

In [ ]:
# Retrieving features for the test data
test_dataset_df = fs.retrieve_feature_values(
    spine_df=spine_df_test, 
    features=[kaggle_fv,custom_fv], 
    exclude_columns=['SURVIVED']
)
test_dataset_df.show(3)

In [ ]:
# Create and persist predictions from registered model given the retrieved features
predictions = registered_model.run(test_dataset_df, function_name='predict')
predictions.write.save_as_table('TITANIC_TEST_PREDICTIONS', mode='overwrite')

# 
predictions = session.table('TITANIC_TEST_PREDICTIONS')
predictions.show()

# 9 - Score your results!
Kaggle expects a CSV file with two columns:  
* PassengerId
* Survived

After offloading the data as CSV-file, you can download it via Snowflake's UI.

If you don't have a Kaggle Account, you can also evaluate your performance with the following call:  
**session.call('calculate_challenge_score', <path-to-your-csv-file>)**


In [ ]:
# Transform data into format expected by Kaggle
kaggle_submission = predictions.select('PASSENGER_ID','SURVIVED_PREDICTION')
kaggle_submission = kaggle_submission.with_column_renamed(col('PASSENGER_ID'),'"PassengerId"')
kaggle_submission = kaggle_submission.with_column_renamed(col('SURVIVED_PREDICTION'),'"Survived"')
kaggle_submission.show()

# Export Predictions to submission.csv
kaggle_submission.write.csv(
    '@KAGGLE_SUBMISSION/submission.csv', 
    header=True, 
    single=True, 
    format_type_options={"COMPRESSION": "NONE"},
    overwrite=True
)

In [ ]:
session.call('calculate_challenge_score', '@KAGGLE_SUBMISSION/submission.csv')